# Amazon VPC Security
There are various options available for controlling and enhancing the security of resources within a cloud infrastructure. This notebook will explore some of these options, including fundamental VPC security concepts such as Security Groups and nACLs (Network Access Control Lists), as well as more sophisticated network protection solutions such as AWS Network Firewall and AWS Web Application Firewall (WAF).

## Security Groups
>*Security Groups* are collections of rules which control the network traffic in and out of the **resources** they are associated with. Security Groups operate by managing what connections can be made to a resource. This means that Security Groups are stateful, since if a connection is allowed, it will continue to exist bidirectionally until it is closed. In other words, security groups retain the state of a connection throughout the interaction.

These are the important characteristics of Security Groups you need to know:

- **Default Security Groups:** When you create a VPC, a Security Group is created by default, which is set to allow all traffic out of the resources it is associated with and only traffic in from the Security Group itself 

- **Traffic Rules:** Security Groups are designed to permit traffic; they cannot explicitly deny traffic. Instead, any traffic that is not expressly allowed by any rule is implicitly denied.

- **Flexibility:** Security Groups exist separately from their associated resources, so you can modify rules without needing to re-provision the resources.

- **Multi-Resource Scope:** Security Groups can be associated with multiple resources at once, meaning you can modify security rules which then affect many resources

### Security Group Rules
Security Group rules are created as either inbound or outbound rules, where inbound traffic comes from an external source to the instance, and outbound traffic comes from the instance to an external destination. Each rule specifies the following attributes:

- **Type:** The type of traffic (or more specifically, the protocol) you want to allow in or out of your instance. Some common protocol options are listed as options in the dropdown (for example SSH, Postgres, HTTP), but you can also select Custom TCP

- **Protocol:** This will be greyed out and filled in for you, unless you select *Custom Protocol* in the Type dropdown list, in which case you can manually type the protocol yourself
- **Port range:** A specific port number or range of port numbers traffic is allowed to access. Again this might be auto-populated depending on what traffic type you chose. 
- **Source/Destination:** For inbound rules, the source of the traffic. For outbound rules, the destination where you want the traffic to go. This can be either your own IP address, any IP address, a custom IP address which you then enter in the next box, another Security Group, or specific AWS services
- **Description:** An optional description of what the rule is and why it is there

This image displays the creation of a Security Group outbound rule to allow HTTP traffic from any IP address.

<p align="center"> <img src="images/SGs.png" height="245" width="1222"/> </p>

### Hands-On: Configuring Security Groups for a Web Server and Database 

In the following common scenario, a website is hosted on an EC2 instance, which communicates with an RDS instance to store data for the website. 

We need to set up the security groups for both web server and RDS instance so that they can communicate with each other securely. The architecture for this will be a public subnet for the EC2 instance, and the RDS instance replicated across two private subnets. 

To outline the process, the order of steps for creating this infrastructure will be as follows:

1. Create and configure Security Group for the EC2 instance

2. Create and configure Security Group for the RDS instance

2. Create three subnets: one public and two private

4. Provision the EC2 instance, associate with the EC2 Security Group

5. Provision the RDS instance, associate with the RDS Security Group

6. Test the connection

#### Step 1: Create a Security Group for the Web Server

- Log in to the AWS management console and change the region to **Ireland (eu-west-1)**

- Navigate to VPC using the search bar, then select **Security groups** on the sidebar, then **Create security group**

- Add a name and description for your web server Security Group, and select a VPC

- Add an **inbound** rule to allow **SSH** traffic from `18.202.216.48/29`. This is the CIDR block needed to be able to access the instance via EC2 Instance Connect in the eu-west-1 region (for testing purposes later).

- Add an **outbound** rule to allow **All traffic** from **Anywhere-IPv4** to reach the instance. This is so that the web server can serve a website to any device.

- Press **Create security group**

Your EC2 Security Group should look like the following:

<p align="center"> <img src="images/web-server-SG.png" height="383" width="1202"/> </p> 

#### Step 2: Create Security Groups for the RDS Server

- Navigate back to the main Security Groups dashboard and **Create a new security group**

- Enter a name and description for this Security Group. Make sure to select the same VPC that you chose for the first Security Group

- Add an **inbound** rule to allow PostgreSQL traffic to **Custom** then choose the web server Security Group in the next box. Port 5432 is the default port used by PostgreSQL.

- Delete all **outbound** rules

Your RDS Security Group should look like the following:


<p align="center"> <img src="images/rds-SG.png" height="499" width="1280"/> </p>

#### Step 3: Create and Configure Subnets
Revisit the [VPC Components](https://colab.research.google.com/github/AI-Core/Content-Public-dev/blob/main/Content/units/Cloud-and-DevOps/16.%20Networking/5.%20VPC%20Components/Notebook.ipynb) lesson if you need a reminder of how to complete the following process.

- Create one subnet called `public-subnet`

- Create two private subnets called `private-subnet-1` and `private-subnet-2`, in separate Availability Zones. Make sure to choose the same VPC that you have been using throughout the Hands-On so far.

- Create an Internet Gateway and attach it to the VPC

- Create a Route Table and make sure the public subnet CIDR block you just created is routed to the Internet Gateway

#### Step 4: Provision EC2 Instance
Revisit the [Amazon EC2](https://colab.research.google.com/github/AI-Core/Content-Public-dev/blob/main/Content/units/Cloud-and-DevOps/3.%20Essential%20Cloud%20Technology/5.%20Amazon%20EC2/Notebook.ipynb) lesson if you need a reminder of how to complete this step.

Create an EC2 instance with the following specification:

- **AMI:** Ubuntu Server 24.04 LTS (HVM) SSD Volume Type

- **Instance Type:** `t2.micro`
- **Key pair:** Proceed without a key pair
- **VPC:** The VPC you have been using so far
- **Subnet:** `public-subnet`
- **Auto-assign public IP:** Enable
- **Firewall:** Select existing security group, choose your pre-created EC2 Security Group
- Everything else as default

#### Step 5: Provision RDS Instance
Revisit the [Amazon RDS](https://colab.research.google.com/github/AI-Core/Content-Public-dev/blob/main/Content/units/Cloud-and-DevOps/3.%20Essential%20Cloud%20Technology/8.%20Amazon%20RDS%20/Notebook.ipynb#scrollTo=h06VBPFq3ZWx) lesson if you need a reminder of how to complete this step.

Create an RDS instance with the following specification:

- **Engine:** PostgreSQL

- **Templates:** Free tier

- **Credentials management:** Self managed. Enter a password.

- **Instance configuration:** `db.t3.micro`

- **Connectivity:** Don't connect to an EC2 resource

- **Network Type:** IPv4

- **VPC:** Choose the VPC you have been using so far

- **DB subnet group:** Create subnet group

- **Public access:** No

- **VPC security group:** Choose existing. Choose your pre-created RDS Security Group.

- Leave everything else as default

#### Step 6: Test the Connection
- Navigate to the EC2 console, and select your EC2 instance

- Press **Connect**

- From the **EC2 Instance Connect** pane, press **Connect**. Ignore the **insufficient IAM permissions** warning.

- In the console terminal that opens, type `nc -vz [your RDS endpoint] 5432`, which will test the connection to your RDS database

- Your output should look like this:

<p align="center"> <img src="images/nc-test.png" height="272" width="835"/> </p>

## nACLs (Network Access Control Lists)
>*nACLs* are collections of rules which control the network traffic in and out of the **subnets** they are associated with. nACLs evaluate each inbound and outbound packet against these rules, meaning they are stateless: each request and response to a subnet is evaluated exclusively.

These are the important characteristics of nACLs you need to know:

- **nACL-Subnet Relationship:** Each subnet must be associated with one nACL, but you can associate many subnets to one nACL

- **Default nACLs:** New subnets get a default nACL, which permits all inbound and outbound traffic

- **Traffic Rules:** nACLs can be made to explicitly allow or deny traffic

- **Rule Numbers:** nACL rules are numbered. Rules are evaluated in order of their rule number, so lower numbers get a higher precedence. The first matching rule decides whether to allow or deny the traffic.
For example, traffic will be allowed in to an instance if there is a rule of number 100 which allows it, even if there is another rule of number 200 which denies it. 

- **Default nACL Rule:** Each nACL includes a rule whose rule number is an asterisk (*). This rule ensures that if a packet doesn't match any of the other numbered rules, it's denied. You can't modify or remove this rule.

#### nACL Rules
Each nACL rule specifies these attributes:

- **Rule Number:** The order of precedence for which rules to evaluate first

- **Source/Destination:** Since nACLs operate at a subnet level, the source or destination of the traffic must be a CIDR block (potentially another subnet)

- **Allow/Deny:** Whether the rule is to allow or deny traffic

The other fields nACL rule fields **Type**, **Protocol** and **Port Range** are the same as in Security Groups. There is no **Description** for nACL rules.

#### nACLs and Ephemeral Ports
[NOTE: Coming back to this after ports section of TCP/UDP is written]

## Security Groups vs NACLs

| Feature | Security Groups | NACLs |
|---------|-----------------|-------|
| Statefulness | Stateful | Stateless |
| Rule scope | Resources | Subnets |
| Traffic scope | Applies to connections | Applies to packets |
| OSI Layer | Layers 3 and 4 | Layer 4 |
| Ability to permit/deny | Can only deny | Can permit or deny |
| Precendence order | No order precedence | Evaluated in order of rule numbers |

### Hands-On: Creating and Configuring nACLs for a Web Server and Database

We are going to continue the Hands-On from earlier in this notebook, by adding and configuring custom nACLs for both the EC2 instance subnet and RDS subnets. So far, the subnets have been using default nACLs which are maximally permissive. We are now going to set only the rules necessary for the web server and database to interact with each other, and for the web server to send and receive traffic from the internet.

#### Step 1: Create nACL Outbound Rules for EC2 instance subnet

- Select **Network ACLs** from the VPC console sidebar, then press **Create network ACL**

- Press **Actions**, then **Edit subnet associations**

- On the next screen, select from the list the subnet containing your EC2 instance, then **Save changes**

- Press **Edit inbound rules**, then add the following rules:

    - Allow **All TCP** from the CIDR block of `private-subnet-1`. This is because inbound traffic from RDS instance will return to an ephemeral port range.

    - Allow **All TCP** from CIDR block of `private-subnet-2`

    - Allow **HTTP (80)** from all IPs (`0.0.0.0/0`). The web server needs to be able to receive incoming traffic from anywhere on the web.

    - Allow **HTTPS (443)** from all IPs (`0.0.0.0/0`)

    - Allow **All traffic** from `18.202.216.48/29`, which is the CIDR block we need to be able to use EC2 Instance Connect

- Press **Save changes**. It should look like the following:

<p align="center"> <img src="images/ec2-nacl-inbound.png" height="402" width="1614"/> </p>

#### Step 2: Create nACL Inbound Rules for EC2 instance subnet

- Press **Outbound rules**, then **Edit outbound rules**

- Add the following rules:

    - Allow **PostgreSQL** to the CIDR block of `private-subnet-1`. This is because outbound traffic to the RDS instance will arrive at port 5432.

    - Allow **PostgreSQL** from CIDR block of `private-subnet-2`

    - Allow **HTTP (80)** to all IPs (`0.0.0.0/0`). The web server needs to be able to send outgoing traffic to anywhere on the web.

    - Allow **HTTPS (443)** to all IPs (`0.0.0.0/0`)

    - Allow **All traffic** to `18.202.216.48/29`

- Press **Save changes**. It should look like the following:

<p align="center"> <img src="images/ec2-nacl-outbound.png" height="401" width="1616"/> </p>

#### Step 3: Create nACL Inbound Rules for RDS instance subnet

- Navigate back to **Network ACLs** from the VPC console sidebar, then press **Create network ACL**

- Enter a name and select your custom VPC

- Follow the same process as in Step 1 to associate this nACL with the two private subnets

- Select **Edit inbound rules** and add a rule to allow **PostgreSQL** traffic from the CIDR block for `public-subnet`. Press **Save changes**.

- Select **Edit outbound rules** and add a rule to allow **All TPC** traffic to the CIDR block for `public-subnet`. Press **Save changes**.

- The nACL rules should look like the following:

<p align="center"> <img src="images/rds-nacl.png" height="403" width="1596"/> </p>

### Step 4: Test the Connection

Follow the process you used in the previous Hands-On to connect to the EC2 instance, then run the same `nc` command as before to test the connection between the EC2 and the RDS. If you have configured your nACLs properly, you should receive the same output as before.


## Firewalls  
A *firewall* is a network security device which monitors incoming and outgoing traffic and decides whether to allow or block specific traffic based on a defined set of security rules. This section outlines two of the main firewall services available for AWS.

### AWS Network Firewall
>*AWS Network Firewall* is an intrusion detection and prevention service for Amazon VPCs. AWS Network Firewall is beneficial if your application requires advanced traffic filtering capabilities beyond source and destination IP addresses and ports (which SGs and NACLs manage). It supports complex rule definitions, including domain name filters, protocol-specific checks, and pattern matching that can address sophisticated threats.

It is made up of the following components:

- **Rule Group:** A reusable collection of criteria which decide whether packets in a traffic flow are allowed to pass or drop. For example, if the traffic came from a particular range of IP addresses. Rule groups are either stateless or stateful: stateless rule groups evaluate packets in isolation, while stateful rule groups evaluate them in the context of their traffic flow. 

- **Firewall Policy:** A reusable collection of Rule Groups, along with some policy-level behaviour such as what happens if traffic breaks midstream, and default actions for both stateless and stateful traffic.
- **Firewall:** Connects the Firewall Policy to the VPC. Each Firewall requires at least one Firewall Policy. The Firewall additionally defines settings like how to monitor and log network traffic information.

### AWS Web Application Firewall (WAF)
>*AWS WAF* is a network protection and monitoring service specifically designed for HTTP/HTTPS traffic to and from web application resources. 

It works by defining traffic rules and then associating them with web application resources. A rule describes traffic patterns to look for in requests and to specify the actions to take on matching requests. For example, a specific type of HTTP request that matches a predefined condition, such as a request that includes a particular string in the URL path, a specific IP address, or a certain query parameter. 

The actions to follow matching requests can be:

- Allow the requests to go to the protected resource for processing and response
- Block the requests
- Count the requests
- Run CAPTCHA or challenge checks against requests to verify human users and standard browser use

AWS WAF is made up of the following components:

- **Rules:** Rules contain JSON statements that define the traffic inspection criteria, then the action to take if a request meets the criteria. These are some examples of criteria you can evaluate with AWS WAF Rules:
    - Defining IP addresses or ranges which requests come from
    - The country or location which requests originate from
    - Check for specific known malicious request attributes such as the query string, the request header value or body values

- **Rule Groups:** Reusable collections of rules. You can create your own rule groups or use predefined ones from the AWS Marketplace.
- **Web ACLs (Access Control Lists):** Collections of rule groups, where you decide in which order the rules and rule groups should be evaluated

## Key Takeaways
- Security Groups allow you to control network traffic to and from specific resources. They are stateful, and designed to explicitly permit traffic, not deny.

- NACLs allow you to control network traffic at the subnet level. They are stateless and can allow or deny traffic.
- SG and NACL rules are defined as either inbound or outbound, specifying the protocol type, port range, source/destination, and a description. Additionally, NACL rules are numbered, and are evaluated in order of their numbers.
- AWS Network Firewall is an intrusion detection and monitoring service for VPCs. It consists of traffic rules, grouped together into Rule Groups, which are then assigned to a Firewall Policy.
- AWS WAF is a network detection and monitoring service for HTTP/HTTPS traffic to and from web application resources. A WAF consists of a web ACL, which is a collection of rule groups. 